<a href="https://colab.research.google.com/github/Lovette-Duke/web_scraping_projects/blob/main/scrape_jobslah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,027 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,592 kB

In [ ]:
!pip install pynput

E: Unable to locate package pynput


In [ ]:
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import time

In [ ]:
def web_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--verbose')
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=options)
  return driver

In [ ]:
link = "https://www.jobslah.com/jobsearch/singapore-jobs"
#<div _ngcontent-ng-c2197434926="" class="card cursor-pointer rounded-2x mb-2 ng-star-inserted" id="jobpostsidecard10"><div _ngcontent-ng-c2197434926=""
#class="card-body p-2">
#<div _ngcontent-ng-c2197434926="" class="row g-2">
#<div _ngcontent-ng-c2197434926="" class="col col-auto"><span _ngcontent-ng-c2197434926="" class="avatar avatar-sm rounded-circle shadow bg-white">
#<img _ngcontent-ng-c2197434926="" loading="lazy" alt="" class="img-fluid ng-star-inserted"
#src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAJYAAACWCAMAAAAL34HQAAAAGXRFWHRTb2Z0d2FyZQBBZG9iZSBJbWFnZVJlYWR5ccllPAAAAFpQTFRF////4uLi5eXl4+Pj7u7u5OTk6+vr5+fn5ubm6enp6Ojo7e3t6urq7Ozs+fn57+/v+vr6+Pj48PDw8fHx+/v7/v7+/Pz88/Pz9vb29/f39PT0/f398vLy9fX1z9zSPQAABRhJREFUeNrs21ubgioUBuAZNRDENEs7WP//b+5cCwsIjbbItPfDdzUXmW+oi5Pz8xMTExMTExMTExMT87/PhW5ck2RtKNX195PwQK7u97Ncw7COH7L4d7JoSNYmT94lT8K3VvPTvs0hPMvlRm7Ds/bOz2xkRVZkRVZk/ZdYJzjZ8TtY9e2CORVwMnK6qDm3f8E6s3ejqqSxHLYyy2n0Thsz1bqs0++SrMVq069kXX6/klXCt+dZlU2lKv+AJeDbt7OfIeFZeMrD7GfwocuOWvYBWPOtlcFn6pB1y511Clnl/58s9o2sNP171vbllk+/gbXfaTkNqlQEYe1zyNHGMjOoUnrSc1ay88bCUeBv5cBKn1FWU8dlJgipPbGwnptj+cxRleiqPOd9YJaLilJ6DcpK51T5U0ULr6zGxuIEs3FX0cwPK5thjZ8RwCHmqExmHOpnPlkHrEmdjXXWWN2b7+2BVYXrfJC1ffO9h0HFv45VD6qvZPEgrLEvIY4s7o/Vdv09XWtj3XqM1lrb8/0JOeNUoMfnBQ+uuUdWg8XIWk7TBMsTlit5BIVahQXlhuWq98+aq1t6EZVHMGDhzzhhEV2BNdP5GJVdHiF01vD4KSy2OsvsbzTWVbKozmKrs156QfUi5rK1oLA/LyLzxpq4t5AzbiSarZXn2FpnqrYWY2zt1gLWrYX60AqV1R26rjugpB3+7mRnefDJ6uvDPXVvYz3Kqcqa7hN9siaqPLDOKitxYzVhWYkTS4iwrMSJJcKy4IF8zxL+WF0N6Sysx01eurGER9Z03dpsWAkpqMqqt0PwZ/Twd936Z1VTrJcJzljloYTKKg/9DetXY72U09dplzyCAEtWeVCpLLJfk1VZJoMa6yhZg0o8WSQQS5miqiwuWUxnEW8s+y1fWdYYFBbnV4X1uIjEHwsHzfbWOm4xIldYAiY4V0trEY+sHsYA5hQDWMm4WFWorINS53osej/+WRO7GHAFxypfAuvdiloYVmKwqBvrui4rMVjUjVWuy0oMFnVjlb5YNSwVn4wNs8ZgUTdW6Y2FQ5i0mWMV1I1V+mNV1sk+snY2FhYUOcXo1ClG6ZHVTLLy3MoqoIRi57ODbpD0AVm5weKSVSqsM6hWZF1NVq6z7v2N1loXbC0oVeUKLBweb8xbPtdZXGcJsTZrT5Xx0xSL6yyhs5SLWBQLWfOPu87KVBaBCY4cQRisYikrdWBRK2u7Mxcpt3KKMagWse6303sW1VnsbTktFrJSFxbVWWx1VjrDakcW1VksJAtXYTnO9jDYDzZUZ7GgrN3rBOemlI0Hi4VlbV8nOCcLq/oTljbtUlj8T1n6JuqTxXWWcGMdPbHuHrzpuc7iOku4sbJlrM3IGnYk8B9TbhqL6yzhxMqyRax7I0nWc0fi5wwdjmRxjQVLRO9Z2TLW5sFSNkrkvtICVraMtZlk0SWsbhmrnGJBrXqyGNNZh69gMYNFwrLGbSWDxQwWCcvKDRZ21XtmsEgYViJZuc7iHFlXnaUMjafTLmclCfz2Wn3T7QyVXWU9Jm24cnU8Xqy5YY6LWfdbqn6yHq3FdZbQWbDyITcQ1OgvBH3MwnEAw73mZOhOCC6KcjyXgFpFivtJMwK9YCHfPCrdVVm1xzjz2PQrari3q49ThQwxm2pW9cxn1d3+Mph/VfMZK5DKnSVmVNyuIvOqbCbOe9bla9s8JzxWjcViewvPmn+9EB4TExMTExMTExMTE+OYfwQYANBtmWye3RQaAAAAAElFTkSuQmCC"><!---->
#</span></div>
#<div _ngcontent-ng-c2197434926="" class="col">
#<div _ngcontent-ng-c2197434926="" class="row g-0">
#<div _ngcontent-ng-c2197434926="" class="col">
#<h4 _ngcontent-ng-c2197434926="" class="font-ave text-16 text-dark text-wrap mb-1" title="SALES MANAGER">SALES MANAGER </h4>
#<div _ngcontent-ng-c2197434926="" class="avenir text-14 text-wrap" title="KINGS INDIAN EXPRESS PTE. LTD.">KINGS INDIAN EXPRESS PTE. LTD.</div>
#</div><!----></div></div></div>
#<div _ngcontent-ng-c2197434926="" class="row g-2">
#<div _ngcontent-ng-c2197434926="" class="col">
#<div _ngcontent-ng-c2197434926="" class="description">
#<span _ngcontent-ng-c2197434926="" class="text-13">Full Time</span><span _ngcontent-ng-c2197434926="" class="text-13">On-site</span>
#<span _ngcontent-ng-c2197434926="" class="text-13">Singapore</span></div></div></div>
#<div _ngcontent-ng-c2197434926="" class="row g-0 align-items-center"><div _ngcontent-ng-c2197434926="" class="col"><div _ngcontent-ng-c2197434926="" class="description"><span _ngcontent-ng-c2197434926="" class="text-13 text-dark">2 days ago</span><!----></div></div><div _ngcontent-ng-c2197434926="" class="col col-auto text-end"><!----><div _ngcontent-ng-c2197434926="" class="text-13 text-primary fw-bold d-flex align-items-center ng-star-inserted"><span class="dib f5 fw4 black-80 pr1"></span><span class="dib f2-5 fw6 black-80"><div class="lh-solid"><span class="dib">SGD 6,000</span><span class="dib"><span class="f5 fw4 ph1"> - </span>10,000</span></div></span><span class="ttc dib f5 fw4 black-60 pr1 i pb"></span></div><!----></div></div></div></div>

<nav _ngcontent-ng-c3582658524="" aria-label="Page navigation"><nav _ngcontent-ng-c3582658524="" aria-label="Page navigation" class="ng-star-inserted"><ul _ngcontent-ng-c3582658524="" class="pagination justify-content-center custom-pagination mt-3"><li _ngcontent-ng-c3582658524="" class="page-item"><a _ngcontent-ng-c3582658524="" tabindex="-1" class="page-link rounded"><i _ngcontent-ng-c3582658524="" class="fa-solid fa-angle-double-left fa-lg"></i></a></li><li _ngcontent-ng-c3582658524="" class="page-item"><a _ngcontent-ng-c3582658524="" tabindex="-1" class="page-link rounded"><i _ngcontent-ng-c3582658524="" class="fa-solid fa-angle-left fa-lg"></i></a></li><li _ngcontent-ng-c3582658524="" aria-current="page" class="page-item ng-star-inserted"><a _ngcontent-ng-c3582658524="" class="page-link rounded">1</a></li><li _ngcontent-ng-c3582658524="" aria-current="page" class="page-item active ng-star-inserted"><a _ngcontent-ng-c3582658524="" class="page-link rounded">2</a></li><li _ngcontent-ng-c3582658524="" aria-current="page" class="page-item ng-star-inserted"><a _ngcontent-ng-c3582658524="" class="page-link rounded">3</a></li><!----><li _ngcontent-ng-c3582658524="" class="page-item"><a _ngcontent-ng-c3582658524="" class="page-link rounded"><i _ngcontent-ng-c3582658524="" class="fa-solid fa-angle-right"></i></a></li><li _ngcontent-ng-c3582658524="" class="page-item"><a _ngcontent-ng-c3582658524="" tabindex="-1" class="page-link rounded"><i _ngcontent-ng-c3582658524="" class="fa-solid fa-angle-double-right fa-lg"></i></a></li></ul></nav><!----></nav>

In [ ]:
driver = web_driver()
driver.get("https://www.jobslah.com/jobsearch/singapore-jobs")
time.sleep(5)

divs = driver.find_elements(By.CSS_SELECTOR, 'div.card.cursor-pointer')

for div in divs:
  # Get Job Tittle
  try:
    span_tag = div.find_element(By.CSS_SELECTOR, 'span.text-13.text-dark').text #<span _ngcontent-ng-c2197434926="" class="text-13 text-dark">2 days ago</span>
    # extract the time text from the span tag
    # posted_text = span_tag.text.strip()  # e.g., "Posted 3h ago"
    posted_dates.append(span_tag)
  except:
    posted_dates.append(None)

driver.quit()
# divs
posted_dates

['22 minutes ago',
 '1 hour ago',
 '1 hour ago',
 '2 hours ago',
 '2 hours ago',
 '2 hours ago',
 '2 hours ago',
 '3 days ago',
 '3 days ago',
 '3 days ago']

In [ ]:
job_titles

[None,
 'Account/HR',
 'Office Admin',
 'Account Admin',
 'Retail Store Manager',
 'Entry Level Sales and Marketing',
 'Junior Marketing Events (Travelling Opportunity)',
 'Tray Assembly (EAST)',
 'MARKETING ASSISTANT MANAGER',
 'MARKETING MANAGER',
 'MANAGEMENT EXECUTIVE']

In [ ]:
job_titles = []
company_names = []
posted_dates = []

In [ ]:
driver = web_driver()
driver.get("https://www.jobslah.com/jobsearch/singapore-jobs")
time.sleep(5)
# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.card.cursor-pointer')))
action = ActionChains(driver)

for page in range(377 ):
  divs = driver.find_elements(By.CSS_SELECTOR, 'div.card.cursor-pointer')
  time.sleep(5)

  for div in divs:
  # Get Job Tittle
    try:
      job_title = div.find_element(By.CSS_SELECTOR, 'h4.font-ave').text
      job_titles.append(job_title)
    except:
      job_titles.append(None)


    # Get Company Name
    try:
      company_name = div.find_element(By.CSS_SELECTOR, 'div.avenir').text
      company_names.append(company_name)
    except:
      company_names.append(None)

    # Get Posted Date
    try:
      posted_date = div.find_element(By.CSS_SELECTOR, 'span.text-13.text-dark').text
      posted_dates.append(posted_date)

    except:
      posted_dates.append(None)
      # print(i)

  # nav = driver.find_element(By.CSS_SELECTOR, 'nav.ng-star-inserted')
  # # <i _ngcontent-ng-c3582658524="" class="fa-solid fa-angle-right"></i>

  # next = nav.find_element(By.CSS_SELECTOR, 'a.page-link.rounded i.fa-solid.fa-angle-right')
  # action.click().perform()
  driver.execute_script("document.querySelector('.fa-angle-right').click()")
  time.sleep(20)


  # except Exception as e:
  #   print(f"Error: {e}")
driver.quit()

In [ ]:
len(job_titles)

3770

In [ ]:
df = pd.DataFrame({
    'Job Title': job_titles,
    'Company Name': company_names,
    'Posted Date': posted_dates
})
df.head(10)

,Job Title,Company Name,Posted Date
0,Events and Marketing Rep,CHAMPION RECRUIT,1 hour ago
1,Junior Sales and Marketing Specialist,Innovation Organization,3 hours ago
2,Junior Marketing,Quartz Global,3 hours ago
3,Cook #Cafe / High End Restaurants #OT #FunEnvi...,Communal Connections Pte Ltd,4 hours ago
4,Chef #Casual #BBQ #Sashimi #Cold Dish #Town #U...,Communal Connections Pte Ltd,4 hours ago
5,Branch Lead,Arty Learning Pte Ltd,5 hours ago
6,English Language Teacher,Arty Learning Pte Ltd,5 hours ago
7,Chief Commercial Officer (Based Overseas) - Te...,CLARIST RESOURCES PTE. LTD.,18 hours ago
8,"Senior Quant Developer (Java, Python, C++) / V...",PINPOINT ASIA INFOTECH PTE. LTD.,18 hours ago
9,C# Developer - Elite Prop Trading Firm,PINPOINT ASIA INFOTECH PTE. LTD.,18 hours ago


In [ ]:
df.to_csv('jobslah_data.csv', index=False)